ReLU after BN

In [1]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.regularizers import l2

from keras.datasets import mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

width, height, channels = X_train.shape[1], X_train.shape[2], 1
X_train = X_train.reshape((X_train.shape[0], width, height, channels))
X_test = X_test.reshape((X_test.shape[0], width, height, channels))

print('Statistics train=%.3f (%.3f), test=%.3f (%.3f)' % (X_train.mean(), X_train.std(), X_test.mean(), X_test.std()))

# one hot encode target values
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
# calculate mean on training dataset
datagen.fit(X_train)

print('Data Generator mean=%.3f, std=%.3f' % (datagen.mean, datagen.std))

# prepare an iterators to scale images
train_iterator = datagen.flow(X_train, y_train, batch_size=64,shuffle=False)
test_iterator = datagen.flow(X_test, y_test, batch_size=64, shuffle=False)
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))

Using TensorFlow backend.


Statistics train=33.318 (78.567), test=33.791 (79.172)
Data Generator mean=33.318, std=78.567
Batches train=938, test=157


In [2]:
model = Sequential()
 
model.add(Convolution2D(16, 3, 3, activation='relu', kernel_regularizer=l2(0.0001), input_shape=(28,28,1))) #26
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Convolution2D(32, 3, 3, activation='relu', kernel_regularizer=l2(0.0001))) #24
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Convolution2D(10, 1, 1, activation='relu',kernel_regularizer=l2(0.0001))) #22
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))#11

model.add(Convolution2D(16, 3, 3, activation='relu',kernel_regularizer=l2(0.0001)))#9
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))


model.add(Convolution2D(16, 3, 3, activation='relu',kernel_regularizer=l2(0.0001)))#7
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))


model.add(Convolution2D(16, 3, 3, activation='relu',kernel_regularizer=l2(0.0001)))#5
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))


model.add(Convolution2D(16, 3, 3, activation='relu',kernel_regularizer=l2(0.0001)))#3
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.1))


model.add(Convolution2D(10, 4, 4))
model.add(BatchNormalization())
model.add(Dropout(0.1))


model.add(Flatten())
model.add(Activation('softmax'))


model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", kernel_regularizer=<keras.reg..., input_shape=(28, 28, 1...)`
  This is separate from the ipykernel package so we can avoid doing imports until


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", kernel_regularizer=<keras.reg...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (1, 1), activation="relu", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", kernel_regularizer=<keras

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 26, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
__________

In [0]:
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
def scheduler(epoch, lr):
  return round(0.003 * 1/(1 + 0.319 * epoch), 10)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.003), metrics=['accuracy'])

In [7]:
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=40,validation_data=(X_test, y_test))
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=1)

Epoch 1/40
938/938 [==============================] - 16s 17ms/step - loss: 0.1207 - acc: 0.9568 - val_loss: 0.2145 - val_acc: 0.9863
Epoch 2/40
938/938 [==============================] - 13s 14ms/step - loss: 0.1172 - acc: 0.9573 - val_loss: 0.3880 - val_acc: 0.9743
Epoch 3/40
938/938 [==============================] - 13s 14ms/step - loss: 0.1157 - acc: 0.9568 - val_loss: 0.2835 - val_acc: 0.9819
Epoch 4/40
938/938 [==============================] - 13s 14ms/step - loss: 0.1093 - acc: 0.9583 - val_loss: 0.3748 - val_acc: 0.9745
Epoch 5/40
938/938 [==============================] - 14s 15ms/step - loss: 0.1115 - acc: 0.9569 - val_loss: 0.4041 - val_acc: 0.9712
Epoch 6/40
938/938 [==============================] - 13s 14ms/step - loss: 0.1112 - acc: 0.9564 - val_loss: 0.7982 - val_acc: 0.9417
Epoch 7/40
938/938 [==============================] - 13s 14ms/step - loss: 0.1090 - acc: 0.9565 - val_loss: 0.5177 - val_acc: 0.9624
Epoch 8/40
938/938 [==============================] - 13s 14ms

In [8]:
print('Test Accuracy: %.3f' % (acc * 100))

Test Accuracy: 99.290
